TESTING

In [1]:
from transformers import BertTokenizer

# Load the tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

def preprocess_email(email):
    # Tokenize the input email text
    encoding = tokenizer(email, truncation=True, padding=True, max_length=512, return_tensors="pt")
    return encoding


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [2]:
from transformers import BertForSequenceClassification
model = BertForSequenceClassification.from_pretrained('phishing-detector')
model.eval()  # Set the model to evaluation mode


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [3]:
import torch

def predict_email_category(email_text):
    # Preprocess the email text
    encoding = preprocess_email(email_text)
    
    # Ensure the input is moved to the correct device (e.g., CUDA or CPU)
    input_ids = encoding['input_ids'].to('cuda') if torch.cuda.is_available() else encoding['input_ids'].to('cpu')
    attention_mask = encoding['attention_mask'].to('cuda') if torch.cuda.is_available() else encoding['attention_mask'].to('cpu')
    
    # Perform inference
    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
    
    # Get the predicted class
    logits = outputs.logits
    predicted_class = torch.argmax(logits, dim=1).item()
    
    # Return the predicted class
    return predicted_class

# Example usage
email_text = "Congratulations! You've won a $1000 gift card. Click here to claim your prize."
predicted_class = predict_email_category(email_text)

# Map predicted class back to label
label_mapping = {0: 'Ham', 1: 'Spam'}
predicted_label = label_mapping[predicted_class]

print(f"Predicted category: {predicted_label}")


Predicted category: Spam


TRAINING ACCURACY - 

Epoch 1
100%|██████████| 279/279 [06:35<00:00,  1.42s/it]
Epoch Loss: 4.522145400548652
Epoch Accuracy: 99.66%
Epoch Loss: 4.522145400548652

Epoch 2
100%|██████████| 279/279 [18:58<00:00,  4.08s/it] 
Epoch Loss: 2.550334491126705
Epoch Accuracy: 99.87%
Epoch Loss: 2.550334491126705

Epoch 3
100%|██████████| 279/279 [06:40<00:00,  1.44s/it]
Epoch Loss: 1.3428259229112882
Epoch Accuracy: 99.91%
Epoch Loss: 1.3428259229112882


In [1]:
import pandas as pd

# Load the dataset
data = pd.read_csv('email.csv')

# Display the first few rows
print(data.head())


from sklearn.preprocessing import LabelEncoder

# Convert categories to numeric labels
label_encoder = LabelEncoder()
data['Category'] = label_encoder.fit_transform(data['Category'])

# Get label mappings
label_mapping = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))
print("Label Mapping:", label_mapping)


from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    data['Message'], data['Category'], test_size=0.2, random_state=42
)

from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')


  Category                                            Message
0      ham  Go until jurong point, crazy.. Available only ...
1      ham                      Ok lar... Joking wif u oni...
2     spam  Free entry in 2 a wkly comp to win FA Cup fina...
3      ham  U dun say so early hor... U c already then say...
4      ham  Nah I don't think he goes to usf, he lives aro...
Label Mapping: {'ham': 0, 'spam': 1, '{"mode":"full"': 2}


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [2]:
def tokenize_texts(texts, labels, tokenizer, max_length=128):
    inputs = tokenizer(
        list(texts),
        max_length=max_length,
        padding='max_length',
        truncation=True,
        return_tensors='pt'
    )
    inputs['labels'] = torch.tensor(labels.tolist())
    return inputs

import torch

train_inputs = tokenize_texts(X_train, y_train, tokenizer)
test_inputs = tokenize_texts(X_test, y_test, tokenizer)


from torch.utils.data import DataLoader, TensorDataset

train_dataset = TensorDataset(
    train_inputs['input_ids'], train_inputs['attention_mask'], train_inputs['labels']
)
test_dataset = TensorDataset(
    test_inputs['input_ids'], test_inputs['attention_mask'], test_inputs['labels']
)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

from transformers import BertForSequenceClassification

model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=3)
model.train()





/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [10]:
from transformers import AdamW
from sklearn.metrics import accuracy_score

optimizer = AdamW(model.parameters(), lr=5e-5)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Training Loop
from tqdm import tqdm

for epoch in range(3):  # Train for 3 epochs
    print(f"Epoch {epoch + 1}")
    epoch_loss = 0
    all_preds = []
    all_labels=[]
    for batch in tqdm(train_loader):
        input_ids, attention_mask, labels = [b.to(device) for b in batch]
        optimizer.zero_grad()
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
        
        logits = outputs.logits
        preds= torch.argmax(logits, dim=-1)
        
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())
    accuracy = accuracy_score(all_labels, all_preds)
    
    print(f"Epoch Loss: {epoch_loss}")
    print(f"Epoch Accuracy: {accuracy * 100:.2f}%")
        
        

    print(f"Epoch Loss: {epoch_loss}")


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1


100%|██████████| 279/279 [06:35<00:00,  1.42s/it]


Epoch Loss: 4.522145400548652
Epoch Accuracy: 99.66%
Epoch Loss: 4.522145400548652
Epoch 2


100%|██████████| 279/279 [18:58<00:00,  4.08s/it] 


Epoch Loss: 2.550334491126705
Epoch Accuracy: 99.87%
Epoch Loss: 2.550334491126705
Epoch 3


100%|██████████| 279/279 [06:40<00:00,  1.44s/it]

Epoch Loss: 1.3428259229112882
Epoch Accuracy: 99.91%
Epoch Loss: 1.3428259229112882


TEST ACCURACY - 

100%|██████████| 70/70 [00:33<00:00,  2.07it/s]
Test Accuracy: 99.10%

In [9]:
from sklearn.metrics import accuracy_score
model.eval()  # Set model to evaluation mode
test_preds = []
test_labels = []

# Evaluation loop
with torch.no_grad():
    for batch in tqdm(test_loader):
        input_ids, attention_mask, labels = [b.to(device) for b in batch]
        
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        
        logits = outputs.logits
        preds = torch.argmax(logits, dim=-1)
        
        test_preds.extend(preds.cpu().numpy())
        test_labels.extend(labels.cpu().numpy())

# Calculate accuracy on the test set
test_accuracy = accuracy_score(test_labels, test_preds)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

100%|██████████| 70/70 [00:33<00:00,  2.07it/s]

Test Accuracy: 99.10%
